# Notebook para demostrar pipeline que utiliza el modelo de regresion logistica Lasso para distinguir entre AI o no

**Cargar librerias**

In [23]:
# Save the model with pkl
import pickle
import gensim
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize
import ripser
from persim import plot_diagrams, PersistenceImager
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Cargar modelo y pipeline completo

In [2]:
# Load the model
with open('log_reg_model.pkl', 'rb') as f:
    model_LR = pickle.load(f)

# Load the keras model
from keras.models import load_model
model_lstm = load_model('lstm_model.keras')


d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### Glove

In [3]:
def load_glove_model(glove_file):
    print("Loading GloVe Model...")
    glove_model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            glove_model[word] = embedding
    print("Done.", len(glove_model), "words loaded!")
    return glove_model

glove_file = 'glove_6B/glove-sbwc.i25.vec'
glove_model = load_glove_model(glove_file)

Loading GloVe Model...
Done. 855380 words loaded!


Funcion para hacer pruebas

In [48]:
def prueba(ejemplo):
    print("Ejemplo:", ejemplo)

    
    sentences = sent_tokenize(ejemplo)


    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]


    def sentence_embedding(sentence_tokens, embeddings, stop_words):
        embedding_dim = 300  # GloVe 50d
        sentence_vector = np.zeros(embedding_dim)
        word_count = 0
        
        for word in sentence_tokens:
            word = word.lower()
            if word not in stop_words and word in embeddings:
                sentence_vector += embeddings[word]
                word_count += 1
                
        if word_count > 0:
            sentence_vector /= word_count  # Optionally, normalize by number of words
        
        return sentence_vector

    embeddings = [sentence_embedding(sentence, glove_model, stopwords.words('spanish')) for sentence in tokenized_sentences]
    embeddings = np.array(embeddings)


    ripserperiod = ripser.ripser(embeddings)["dgms"]
    h0_diagram = ripserperiod[0].copy()
    h0_diagram = h0_diagram[np.isfinite(h0_diagram).all(axis=1)]
    # Generate the images for all the persistence diagrams at once to assure the same pixel size
    # Manually set birth and persistence ranges based on your data

    #Los siguientes parámetros mantienen el tamaño de la imagen de persistencia (matriz)
    birth_range = (0, 0.5)  
    pers_range = (0, 3)   

    # Initialize PersistenceImager
    pimgr = PersistenceImager(pixel_size=0.01, birth_range=birth_range, pers_range=pers_range)
    #pimgr = PersistenceImager(pixel_size=0.1)
    pimgr.kernel_params = {'sigma': 0.01}
    pdgms = h0_diagram#lista_vectores
    #pimgr.fit(lista_vectores, skew=True)
    pimgs = pimgr.transform(pdgms,skew=True)

    '''
    pimgr.plot_image(pimgs)
    plt.tight_layout()
    plt.show()
    '''
    #predicción LASSO
    #flatten the image to a 1D array
    pimgs = pimgs.flatten()
    pred1 = model_LR.predict_proba(pimgs.reshape(1,-1))
    print("Predicción TDA: Humano = ", pred1[0][0]*100, "%   , AI = ", pred1[0][1]*100, "%")

    #predicción LSTM
    max_timesteps = 8
    reshaped_data = embeddings.reshape((1, embeddings.shape[0], 300))
    X_padded = pad_sequences(reshaped_data, maxlen=max_timesteps, dtype='float32', padding='post', truncating='post')
    pred2 = model_lstm.predict(X_padded, verbose=0)
    print("Predicción LSTM: Humano = ", (1-pred2[0][0])*100, "%   , AI = ", pred2[0][0]*100, "%")

    #prediccion final
    w=4
    pred_final=((pred1[0][1]+pred2[0][0]*w)/(1+w))
    print("Predicción final: Humano = ", (1-pred_final)*100, "%   , AI = ", pred_final*100, "%")
    
    

# Aquí se pueden realizar pruebas

In [49]:
caso='Un buen profesor. es alguien que se preocupa por sus estudiantes. Se encarga de hacer que los alumnos se cuestionen acerca de lo que creen que saben'
prueba(caso)

Ejemplo: Un buen profesor. es alguien que se preocupa por sus estudiantes. Se encarga de hacer que los alumnos se cuestionen acerca de lo que creen que saben
Predicción TDA: Humano =  60.62863775273956 %   , AI =  39.37136224726044 %
Predicción LSTM: Humano =  83.85356515645981 %   , AI =  16.14643484354019 %
Predicción final: Humano =  79.20857967571575 %   , AI =  20.791420324284243 %


d:\miniconda\envs\gcwtdaENV\lib\site-packages\ripser\ripser.py:253: UserWarning: The input point cloud has more columns than rows; did you mean to transpose?
  warnings.warn(


In [50]:
caso='Un buen profesor necesita pasión por enseñar, dominio de su materia y habilidades de comunicación que le permitan explicar conceptos de forma clara y accesible. Además, debe ser paciente y empático, comprendiendo que cada estudiante aprende a su propio ritmo. Su capacidad para motivar es clave, inspirando curiosidad y ganas de aprender en sus alumnos. También debe ser flexible y creativo, adaptando sus estrategias según las necesidades del grupo, y practicar la escucha activa para atender dudas y preocupaciones. La evaluación y la retroalimentación efectiva son esenciales para detectar fortalezas y áreas de mejora, ofreciendo sugerencias útiles. Un buen profesor utiliza recursos variados, como herramientas tecnológicas y actividades interactivas, para enriquecer el aprendizaje. Por último, su compromiso con la formación continua le permite mejorar constantemente su metodología y mantenerse actualizado en su disciplina.'
prueba(caso)

Ejemplo: Un buen profesor necesita pasión por enseñar, dominio de su materia y habilidades de comunicación que le permitan explicar conceptos de forma clara y accesible. Además, debe ser paciente y empático, comprendiendo que cada estudiante aprende a su propio ritmo. Su capacidad para motivar es clave, inspirando curiosidad y ganas de aprender en sus alumnos. También debe ser flexible y creativo, adaptando sus estrategias según las necesidades del grupo, y practicar la escucha activa para atender dudas y preocupaciones. La evaluación y la retroalimentación efectiva son esenciales para detectar fortalezas y áreas de mejora, ofreciendo sugerencias útiles. Un buen profesor utiliza recursos variados, como herramientas tecnológicas y actividades interactivas, para enriquecer el aprendizaje. Por último, su compromiso con la formación continua le permite mejorar constantemente su metodología y mantenerse actualizado en su disciplina.
Predicción TDA: Humano =  9.86844183525023 %   , AI =  90.

d:\miniconda\envs\gcwtdaENV\lib\site-packages\ripser\ripser.py:253: UserWarning: The input point cloud has more columns than rows; did you mean to transpose?
  warnings.warn(
